In [199]:
import numpy as np
import re

In [200]:
f = open("../input/day7.txt","r")
lines = f.read().splitlines()
len(lines)

1040

In [201]:
lines_test = ["$ cd /",
"$ ls",
"dir a",
"14848514 b.txt",
"8504156 c.dat",
"dir d",
"$ cd a",
"$ ls",
"dir e",
"29116 f",
"2557 g",
"62596 h.lst",
"$ cd e",
"$ ls",
"584 i",
"$ cd ..",
"$ cd ..",
"$ cd d",
"$ ls",
"4060174 j",
"8033020 d.log",
"5626152 d.ext",
"7214296 k",]

In [202]:
from anytree import NodeMixin, RenderTree
class FSNode(NodeMixin):  
    def __init__(self, name, size=None, parent=None):
        self.name = name
        self.size = size
        self.parent = parent

In [203]:
DIR_REGEX = r"dir (\w+)"
FILE_REGEX = r"(\d+) ((\w|\.)+)"
CD_FOLDER = r"\$ cd (\w+)"
CD_PARENT = r"\$ cd \.\."

In [204]:
root = FSNode('/')
current = root

for line in lines:
    if re.match(r"\$ cd \/", line):
        current = root
        continue
    if re.match(r"\$ ls", line):
        continue
    if re.match(DIR_REGEX, line):
        dirname = re.match(DIR_REGEX, line).group(1)
        FSNode(dirname, parent=current)
    if re.match(FILE_REGEX, line):
        filesize, filename,_  = re.match(FILE_REGEX, line).groups()
        FSNode(filename, filesize, parent=current)
    if re.match(CD_FOLDER, line):
        dirname = re.match(CD_FOLDER, line).group(1)
        child_index = [x.name for x in current.children].index(dirname)
        current = current.children[child_index]
    if re.match(CD_PARENT, line):
        current = current.parent


In [205]:
at_most_100000_sum = 0
def calc_size(node: FSNode):
    sum = 0
    for child in node.children:
        child: FSNode = child
        if child.is_leaf:
            sum += int(child.size)
        else:
            sum += calc_size(child)
            
    node.size = sum
    if sum < 100000 and not node.is_root:
        global at_most_100000_sum
        at_most_100000_sum += sum;

    return node.size

In [206]:
calc_size(root)
print()
# print(at_most_100000_sum)

In [207]:
for pre, fill, node in RenderTree(root):
    treestr = u"%s%s" % (pre, node.name)
    # print(treestr.ljust(8), node.size)

In [208]:
capacity = 70000000
required = 30000000
free = capacity - root.size
missing = required - free
# missing

In [209]:
def find_best_folder(node: FSNode):
    if node.size < missing:
        return None

    best = node.size
    for child in node.children:
        child: FSNode = child
        if not child.is_leaf and child.size > missing:
            best_folder = find_best_folder(child)
            if best_folder and best_folder < best:
                best = best_folder

    return best

In [210]:
# find_best_folder(root)